In [1]:
import pandas as pd
from numba import jit
import numpy as np

In [2]:
station_data = pd.read_csv("datasets/station.csv")

### Compilation Optimizations

So far, we have optimized the data dtypes to reduce the memory usage, parallelized the computation, but they say [python is inherently slow](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/)

- **Python is intrepreted not compiled.**
    - Not knowing what's ahead in the code takes away the possibility of optimization ahead of computation.
    
- **Everything in python is an object..!**
    - It is dynamically typed and compiler has no idea what the dtype of variable is.
    
- **And we have seen how python lists are stored..!!**

We still use python because 
- It is **flexible** (need not share different execution files for different platforms..!)
- Easy to use and understand
- Reduced debugging time and flexibility leads to better development time
- of course the community..!

There have been several optimizations done for runtime compilations. One such is `jit` where some functions (user specified in most cases) are complied to bytecode and stored for reuse instead of recompiling during runtime. 

One such compiler is `LLVM` which Numba uses to optimize compilation. You can checkout the functions and dtypes hat are supported on Numba [here](https://numba.pydata.org/numba-doc/dev/reference/numpysupported.html).

In [3]:
lat_long = station_data[['lat','long']]

In [4]:
lat_long_list = lat_long.values.tolist()

In [5]:
lat_long_array = np.asarray(lat_long_list)

In [6]:
def pairwise_python(array):
    m = array.shape[0]
    n = array.shape[1]
    D = np.empty((m, m), dtype=np.float)
    for i in range(m):
        for j in range(m):
            d = 0.0
            for k in range(n):
                tmp = array[i, k] - array[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [7]:
@jit
def pairwise_numba(array):
    m = array.shape[0]
    n = array.shape[1]
    D = np.empty((m, m), dtype=np.float)
    for i in range(m):
        for j in range(m):
            d = 0.0
            for k in range(n):
                tmp = array[i, k] - array[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D

In [8]:
%%time
a = pairwise_python(lat_long_array)

CPU times: user 14 ms, sys: 287 µs, total: 14.3 ms
Wall time: 14.2 ms


In [9]:
%%time
opt_a = pairwise_numba(lat_long_array)

CPU times: user 272 ms, sys: 13.6 ms, total: 286 ms
Wall time: 294 ms


In [10]:
from numpy.random import random as randvec

In [11]:
def get_rand_vec(n):
    return randvec((1000, 3))

In [12]:
%%time
a = pairwise_python(get_rand_vec(1000))

CPU times: user 3.25 s, sys: 9.86 ms, total: 3.26 s
Wall time: 3.26 s


In [13]:
%%time 
a_opt = pairwise_numba(get_rand_vec(1000))

CPU times: user 6.67 ms, sys: 2.94 ms, total: 9.61 ms
Wall time: 9.34 ms


In [14]:
%%time
a = pairwise_python(get_rand_vec(3000))

CPU times: user 3.19 s, sys: 10.8 ms, total: 3.21 s
Wall time: 3.21 s


In [15]:
%%time 
a_opt = pairwise_numba(get_rand_vec(3000))

CPU times: user 4.82 ms, sys: 73 µs, total: 4.89 ms
Wall time: 4.86 ms


### Scope for optimization

We can see that this scales really well for us. This can be further optimised, by specifying the data types which are passed to function the dynamic typed issue with python.

The data types can be passed as 
 - **@jit**( `return dtype` ( `input dtypes` ) )
 
This sort of compilation is called `Eager Compilation` where we pass the data types as opposed to `lazy compilation` where the dtypes are passed everytime.


Moreover one `jit` decorated function can be called in another `jit` decorated function 


In [16]:
from numba import float64

In [17]:
@jit(float64(float64, float64))
def sqr_dist(coord1, coord2):
    return (coord1 - coord2)**2

In [18]:
@jit
def pairwise_numba_with_dist(array):
    m = array.shape[0]
    n = array.shape[1]
    D = np.empty((m, m), dtype=np.float)
    for i in range(m):
        for j in range(m):
            d = 0.0
            for k in range(n):
                d += sqr_dist(array[i, k] , array[j, k])
            D[i, j] = np.sqrt(d)
    return D

In [19]:
%%time
opt_a = pairwise_numba(lat_long_array)

CPU times: user 60 µs, sys: 1 µs, total: 61 µs
Wall time: 64.8 µs


In [20]:
%%time
opt_b = pairwise_numba_with_dist((lat_long_array))

CPU times: user 205 ms, sys: 3.84 ms, total: 209 ms
Wall time: 208 ms


In [21]:
%%time 
a_opt = pairwise_numba(get_rand_vec(1000))

CPU times: user 5.37 ms, sys: 120 µs, total: 5.49 ms
Wall time: 5.46 ms


In [22]:
%%time 
a_opt = pairwise_numba_with_dist(get_rand_vec(1000))

CPU times: user 5.4 ms, sys: 529 µs, total: 5.93 ms
Wall time: 5.38 ms


In [23]:
%%time 
a_opt = pairwise_numba(get_rand_vec(5000))

CPU times: user 5.16 ms, sys: 386 µs, total: 5.55 ms
Wall time: 5.17 ms


In [24]:
%%time 
a_opt = pairwise_numba_with_dist(get_rand_vec(5000))

CPU times: user 5.1 ms, sys: 54 µs, total: 5.15 ms
Wall time: 5.08 ms


### Any further optimization ?

We didnot optimise the initial `jit` funtion where we passed array. 

Arrays can be passed to git as `dtype[:]`. Example a flaot array would be passed as `float32[:]`

In [25]:
@jit(float64[:](float64[:]))
def pairwise_numba_with_dist_overall(array):
    m = array.shape[0]
    n = array.shape[1]
    D = np.empty((m, m), dtype=np.float)
    for i in range(m):
        for j in range(m):
            d = 0.0
            for k in range(n):
                d += sqr_dist(array[i, k] , array[j, k])
            D[i, j] = np.sqrt(d)
    return D


In [26]:
%%time 
a_opt = pairwise_numba_with_dist(get_rand_vec(5000))

CPU times: user 5.41 ms, sys: 536 µs, total: 5.95 ms
Wall time: 5.42 ms


In [27]:
%%time 
a_opt = pairwise_numba_with_dist_overall(get_rand_vec(5000))

CPU times: user 121 ms, sys: 3.31 ms, total: 125 ms
Wall time: 123 ms


In [33]:
%%time 
a_opt = pairwise_numba_with_dist(get_rand_vec(10000))

CPU times: user 4.84 ms, sys: 458 µs, total: 5.3 ms
Wall time: 4.86 ms


In [34]:
%%time 
a_opt = pairwise_numba_with_dist_overall(get_rand_vec(10000))

CPU times: user 4.87 ms, sys: 195 µs, total: 5.07 ms
Wall time: 4.9 ms


In [35]:
%%time
a = pairwise_python(get_rand_vec(10000))

CPU times: user 3.23 s, sys: 4.5 ms, total: 3.23 s
Wall time: 3.23 s


We have reduced the compilation time from ~3.5 seconds for 10000 data points to ~5 ms. Incredible 500x reduction in compute time.!!!

There are further optimizations which can be done on top of this. Can check out the workings [here](https://numba.pydata.org/numba-doc/dev/user/jit.html)